# DeepFM Recommender System - Expedia Hotel dataset

## Import Libraries

In [1]:
# conda install seaborn

In [2]:
# pip install holidays

In [3]:
# pip install deepctr

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import tensorflow as tf
import warnings
import math
import sys
import holidays
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import roc_auc_score, recall_score, precision_score, f1_score

from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from math import sqrt


from tensorflow.python.keras.models import Model
from tensorflow.python.keras.layers import Dense

# from deepctr.inputs import build_input_features, get_linear_logit, input_from_feature_columns, combined_dnn_input
from deepctr.layers.core import PredictionLayer, DNN
from deepctr.layers.utils import add_func
from deepctr.models import DeepFM


from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# from deepctr.inputs import SparseFeat,get_feature_names
from deepctr.feature_column import SparseFeat,get_feature_names

In [3]:
df = pd.read_csv('train.csv', sep=',', nrows=100000)
df.shape

(100000, 24)

## Data Preprocessing

In [4]:
df.head(n=2)

,date_time,site_name,posa_continent,user_location_country,user_location_region,user_location_city,orig_destination_distance,user_id,is_mobile,is_package,...,srch_children_cnt,srch_rm_cnt,srch_destination_id,srch_destination_type_id,is_booking,cnt,hotel_continent,hotel_country,hotel_market,hotel_cluster
0,2014-08-11 07:46:59,2,3,66,348,48862,2234.2641,12,0,1,...,0,1,8250,1,0,3,2,50,628,1
1,2014-08-11 08:22:12,2,3,66,348,48862,2234.2641,12,0,1,...,0,1,8250,1,1,1,2,50,628,1


In [5]:
# rename 2 columns
df = df.rename(columns={'hotel_cluster': 'item_id', 'is_booking': 'rating'})

In [6]:
# len(df['user_location_country'].unique())

In [7]:
# df.info()

## Feature Engineering

In [8]:
df= df.drop(['orig_destination_distance'],axis=1)
df = df.dropna()

In [9]:
# df.info()

In [10]:
# Define holidays in some countries
ca_holidays = holidays.Canada()
us_holidays = holidays.UnitedStates()

uk_holidays = holidays.UnitedKingdom()
gr_holidays = holidays.Germany()

In [11]:
# check if checkin or checkout date is in holiday of different countries
df['north_am_ci'] = df['srch_ci'].apply(lambda x: 1 if x in (us_holidays or ca_holidays)  else 0)
df['north_am_co'] = df['srch_co'].apply(lambda x: 1 if x in (us_holidays or ca_holidays)  else 0)

df['europe_ci'] = df['srch_ci'].apply(lambda x: 1 if x in (uk_holidays or gr_holidays)  else 0)
df['europe_co'] = df['srch_co'].apply(lambda x: 1 if x in (uk_holidays or gr_holidays)  else 0)

In [12]:
df.head(n=2)

,date_time,site_name,posa_continent,user_location_country,user_location_region,user_location_city,user_id,is_mobile,is_package,channel,...,rating,cnt,hotel_continent,hotel_country,hotel_market,item_id,north_am_ci,north_am_co,europe_ci,europe_co
0,2014-08-11 07:46:59,2,3,66,348,48862,12,0,1,9,...,0,3,2,50,628,1,0,0,0,0
1,2014-08-11 08:22:12,2,3,66,348,48862,12,0,1,9,...,1,1,2,50,628,1,0,0,0,0


In [13]:
# extract month from date_time
df['click_month'] = pd.DatetimeIndex(df['date_time']).month
df['checkin_month'] = pd.DatetimeIndex(df['srch_ci']).month
df['checkout_month'] = pd.DatetimeIndex(df['srch_co']).month

In [14]:
df= df.drop(['date_time'],axis=1)
df= df.drop(['srch_ci'],axis=1)
df= df.drop(['srch_co'],axis=1)

In [15]:
df.shape

(99929, 27)

In [16]:
df.columns

Index(['site_name', 'posa_continent', 'user_location_country',
       'user_location_region', 'user_location_city', 'user_id', 'is_mobile',
       'is_package', 'channel', 'srch_adults_cnt', 'srch_children_cnt',
       'srch_rm_cnt', 'srch_destination_id', 'srch_destination_type_id',
       'rating', 'cnt', 'hotel_continent', 'hotel_country', 'hotel_market',
       'item_id', 'north_am_ci', 'north_am_co', 'europe_ci', 'europe_co',
       'click_month', 'checkin_month', 'checkout_month'],
      dtype='object')

In [17]:
#Remove rows with the same user_id and item_id and different rating

#max_rating = df.groupby(['user_id', 'item_id']).rating.transform(max)
#df = df.loc[df.rating == max_rating]
#df.drop_duplicates(keep='first',inplace=True)

In [18]:
# categ_sparse / conti_dense
sparse_features = ['site_name','posa_continent','user_location_country','user_location_region','user_location_city',
             'user_id','is_mobile','is_package','channel','click_month','checkin_month','checkout_month',
            'srch_adults_cnt','srch_children_cnt','srch_rm_cnt','srch_destination_id','hotel_continent',
               'hotel_country','cnt','north_am_ci', 'north_am_co', 'europe_ci', 'europe_co','click_month',
                   'checkin_month','checkout_month']

dense_features = ['hotel_market']
target = ['rating']

### Simple preprocessing

In [19]:
# Label Encoding for sparse features,and normalization for dense numerical features
for feat in sparse_features:
    lbe = LabelEncoder()
    df[feat] = lbe.fit_transform(df[feat])

In [20]:
mms = MinMaxScaler(feature_range=(0,1))
df[dense_features] = mms.fit_transform(df[dense_features])

### Generate feature columns
For sparse features, we transform them into dense vectors by embedding techniques. For dense numerical features, we concatenate them to the input tensors of fully connected layer.

In [21]:
# count #unique features for each sparse field
fixlen_feature_columns = [SparseFeat(feat, df[feat].nunique(),embedding_dim=4)
                          for feat in sparse_features]
linear_feature_columns = fixlen_feature_columns
dnn_feature_columns = fixlen_feature_columns
feature_names = get_feature_names(linear_feature_columns + dnn_feature_columns)

### Generate the training samples and train the model

In [22]:
# generate input data for model
train, test = train_test_split(df, test_size=0.2)
train_model_input = {name:train[name].values for name in feature_names}
test_model_input = {name:test[name].values for name in feature_names}

# Basic DeepFM (with Best Hyper-parameter)

In [23]:
# model = DeepFM(linear_feature_columns,dnn_feature_columns,task='binary')
model = DeepFM(linear_feature_columns, dnn_feature_columns, dnn_hidden_units=(128,128), dnn_activation='relu',task='binary')
model.compile("adam", "mse", metrics=['mse'])

history = model.fit(train_model_input, train[target].values,
                    batch_size=20, epochs=10, verbose=2, validation_split=0.2, )
pred_ans = model.predict(test_model_input)

Train on 63954 samples, validate on 15989 samples
Epoch 1/10


/scratch/x2093a05/.conda/envs/notebook/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/scratch/x2093a05/.conda/envs/notebook/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


63954/63954 - 67s - loss: 0.0719 - mse: 0.0712 - val_loss: 0.0708 - val_mse: 0.0697
Epoch 2/10
63954/63954 - 57s - loss: 0.0674 - mse: 0.0659 - val_loss: 0.0714 - val_mse: 0.0697
Epoch 3/10
63954/63954 - 57s - loss: 0.0659 - mse: 0.0639 - val_loss: 0.0733 - val_mse: 0.0712
Epoch 4/10
63954/63954 - 57s - loss: 0.0646 - mse: 0.0624 - val_loss: 0.0744 - val_mse: 0.0721
Epoch 5/10
63954/63954 - 57s - loss: 0.0637 - mse: 0.0613 - val_loss: 0.0757 - val_mse: 0.0732
Epoch 6/10
63954/63954 - 57s - loss: 0.0627 - mse: 0.0602 - val_loss: 0.0767 - val_mse: 0.0742
Epoch 7/10
63954/63954 - 57s - loss: 0.0619 - mse: 0.0593 - val_loss: 0.0785 - val_mse: 0.0759
Epoch 8/10
63954/63954 - 56s - loss: 0.0610 - mse: 0.0584 - val_loss: 0.0805 - val_mse: 0.0778
Epoch 9/10
63954/63954 - 56s - loss: 0.0603 - mse: 0.0575 - val_loss: 0.0820 - val_mse: 0.0792
Epoch 10/10
63954/63954 - 57s - loss: 0.0593 - mse: 0.0565 - val_loss: 0.0820 - val_mse: 0.0792


In [24]:
basic_auc = roc_auc_score(test[target].values, pred_ans)

print("RMSE:\t%f" % np.round(math.sqrt(mean_squared_error(test[target].values, pred_ans)),3),
      "MAE:\t%f" % np.round(mean_absolute_error(test[target].values, pred_ans),3),
      "MSE:\t%f" % np.round(mean_squared_error(test[target].values, pred_ans),3),
      "AUC:\t%f" % np.round(basic_auc,3),
      sep='\n')

RMSE:	0.284000
MAE:	0.136000
MSE:	0.081000
AUC:	0.763000


<h1>Federated Lerning</h1>

In [25]:
#  데이터 전처리

In [26]:
df = pd.read_csv('train.csv', sep=',', nrows=100000)
df = df.rename(columns={'hotel_cluster': 'item_id', 'is_booking': 'rating'})
df= df.drop(['orig_destination_distance'],axis=1)
df = df.dropna()

df['north_am_ci'] = df['srch_ci'].apply(lambda x: 1 if x in (us_holidays or ca_holidays)  else 0)
df['north_am_co'] = df['srch_co'].apply(lambda x: 1 if x in (us_holidays or ca_holidays)  else 0)

df['europe_ci'] = df['srch_ci'].apply(lambda x: 1 if x in (uk_holidays or gr_holidays)  else 0)
df['europe_co'] = df['srch_co'].apply(lambda x: 1 if x in (uk_holidays or gr_holidays)  else 0)

# extract month from date_time
df['click_month'] = pd.DatetimeIndex(df['date_time']).month
df['checkin_month'] = pd.DatetimeIndex(df['srch_ci']).month
df['checkout_month'] = pd.DatetimeIndex(df['srch_co']).month

df= df.drop(['date_time'],axis=1)
df= df.drop(['srch_ci'],axis=1)
df= df.drop(['srch_co'],axis=1)


In [27]:
# 초기 Global model 학습

In [28]:
df_shuffled = df.sample(frac=1).reset_index(drop=True) # 데이터 프레임 셔플


In [29]:
def feat_transform(df):
    df = df.copy()
    for feat in sparse_features:
        lbe = LabelEncoder()
        df[feat] = lbe.fit_transform(df[feat])
    mms = MinMaxScaler(feature_range=(0,1))
    df[dense_features] = mms.fit_transform(df[dense_features])

    fixlen_feature_columns = [SparseFeat(feat, df[feat].nunique(),embedding_dim=4)
                          for feat in sparse_features]
    linear_feature_columns = fixlen_feature_columns
    dnn_feature_columns = fixlen_feature_columns
#     feature_names = get_feature_names(linear_feature_columns + dnn_feature_columns)
#     train_model_input = {name:df[name].values for name in feature_names}
    
    return df, linear_feature_columns, dnn_feature_columns

In [30]:
def df_to_modelInput(df, linear_feature_columns, dnn_feature_columns):
    feature_names = get_feature_names(linear_feature_columns + dnn_feature_columns)
    train_model_input = {name:df[name].values for name in feature_names}
    return train_model_input

In [31]:
# 전체 데이터에 대해 임베딩 구하기

In [32]:
trans_df , linear_feature_columns, dnn_feature_columns = feat_transform(df_shuffled)

In [33]:
split_size = int(len(trans_df) * 0.2)
# split_size = int(split_size)

In [34]:
init_data = trans_df.iloc[:split_size]
test_data = trans_df.iloc[split_size:split_size*2]
df_shuffled = trans_df.iloc[split_size*2:]

In [35]:
init_model_input = df_to_modelInput(init_data, linear_feature_columns, dnn_feature_columns)
test_model_input = df_to_modelInput(test_data, linear_feature_columns, dnn_feature_columns)

In [40]:
global_model = DeepFM(linear_feature_columns, dnn_feature_columns, dnn_hidden_units=(128,128), dnn_activation='relu',task='binary')
global_model.compile("adam", "mse", metrics=['mse'])

history = global_model.fit(init_model_input, init_data[target].values,
                    batch_size=20, epochs=10, verbose=2, validation_split=0.2, )


Train on 15988 samples, validate on 3997 samples
Epoch 1/10


/scratch/x2093a05/.conda/envs/notebook/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/scratch/x2093a05/.conda/envs/notebook/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


15988/15988 - 20s - loss: 0.0749 - mse: 0.0747 - val_loss: 0.0731 - val_mse: 0.0728
Epoch 2/10
15988/15988 - 13s - loss: 0.0609 - mse: 0.0603 - val_loss: 0.0792 - val_mse: 0.0784
Epoch 3/10
15988/15988 - 13s - loss: 0.0498 - mse: 0.0488 - val_loss: 0.0847 - val_mse: 0.0837
Epoch 4/10
15988/15988 - 14s - loss: 0.0435 - mse: 0.0424 - val_loss: 0.0927 - val_mse: 0.0915
Epoch 5/10
15988/15988 - 14s - loss: 0.0395 - mse: 0.0382 - val_loss: 0.0974 - val_mse: 0.0960
Epoch 6/10
15988/15988 - 13s - loss: 0.0366 - mse: 0.0352 - val_loss: 0.0995 - val_mse: 0.0981
Epoch 7/10
15988/15988 - 14s - loss: 0.0342 - mse: 0.0328 - val_loss: 0.0994 - val_mse: 0.0980
Epoch 8/10
15988/15988 - 13s - loss: 0.0324 - mse: 0.0310 - val_loss: 0.1114 - val_mse: 0.1100
Epoch 9/10
15988/15988 - 13s - loss: 0.0305 - mse: 0.0291 - val_loss: 0.1040 - val_mse: 0.1026
Epoch 10/10
15988/15988 - 13s - loss: 0.0297 - mse: 0.0284 - val_loss: 0.1055 - val_mse: 0.1042


In [41]:
init_weights = global_model.get_weights()

In [42]:
# Local 및 Global 학습

In [44]:
groups = df_shuffled.groupby(df_shuffled.user_id)
# user_id별로 client 구분
clients = []
min_data_num = 20
for user_id in df_shuffled['user_id'].unique():
    if df_shuffled['user_id'].value_counts()[user_id] > min_data_num:
        clients.append(groups.get_group(user_id))

In [48]:
max_clients = int(len(clients) * 0.2)
clients_num = len(clients)
stop_round = clients_num // max_clients
current_round = 0
clt_idx = 0
local_epoch = 1
local_batch_size = 20

In [49]:
print('max_clients : ', max_clients)
print('clients_num : ', clients_num)

max_clients :  163
clients_num :  816


In [50]:
import random 
from tensorflow.keras import backend as K
import numpy as np
# Client sampling
f = open(f'{min_data_num}_{max_clients}_{local_epoch}_log.txt', 'a')

while True:
    if stop_round == current_round:
        f.close()
        break
    if current_round == 0:
        global_weights = init_weights
    else:
        global_model.save_weights(f'global_weights_{min_data_num}_{max_clients}_{local_epoch}_rnd{current_round}.h5')
        global_weights = global_model.get_weights()
    scaled_local_weight_list = list()
    
    if clt_idx < clients_num:
        for client_df in clients[clt_idx:clt_idx+max_clients]:
            train_model_input = df_to_modelInput(client_df, linear_feature_columns, dnn_feature_columns)
            local_model = DeepFM(linear_feature_columns, dnn_feature_columns, dnn_hidden_units=(128,128), dnn_activation='relu',task='binary')
            local_model.compile("adam", "mse", metrics=['mse'])

            local_model.set_weights(global_weights)
            history = local_model.fit(train_model_input, client_df[target].values,
                        batch_size=local_batch_size, epochs=local_epoch, verbose=2)

            scaled_weights = local_model.get_weights()
            scaled_local_weight_list.append(scaled_weights)
            
            K.clear_session()
            
        clt_idx += max_clients
        
        temp_list = []
        
        temp_weight = scaled_local_weight_list[0]
        
        for i in range(len(temp_weight)):
            temp = np.array(temp_weight[i])
            temp_list.append(temp)

        temp_list = np.array(temp_list)
        
        for i in range(len(scaled_local_weight_list)):
            for j in range(len(scaled_local_weight_list[i])):
                temp = np.array(scaled_local_weight_list[i][j])
                temp_list[j] += temp
        average_weights = np.divide(temp_list, max_clients)
        
        global_model.set_weights(average_weights.tolist())
        pred_ans = global_model.predict(test_model_input, batch_size=local_batch_size)
        auc = roc_auc_score(test_data[target].values, pred_ans)

        RMSE = np.round(math.sqrt(mean_squared_error(test_data[target].values, pred_ans)),3)
        MAE = np.round(mean_absolute_error(test_data[target].values, pred_ans),3)
        MSE = np.round(mean_squared_error(test_data[target].values, pred_ans),3)
        auc = np.round(auc,3)
        print(f"current_round : {current_round}")
        print("RMSE:\t%f" % RMSE,
              "MAE:\t%f" % RMSE,
              "MSE:\t%f" % MSE,
              "AUC:\t%f" % auc,
              sep='\n')
        f.write(f"current_round:{current_round}")
        f.write(f"RMSE:{RMSE}")
        f.write(f"MAE:{MAE}")
        f.write(f"MSE:{MSE}")
        f.write(f"AUC:{auc}")
        
        current_round += 1
    else:
        random.shuffle(clients)
        clt_idx = 0

Train on 103 samples


/scratch/x2093a05/.conda/envs/notebook/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/scratch/x2093a05/.conda/envs/notebook/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


103/103 - 8s - loss: 0.0784 - mse: 0.0771
Train on 187 samples


/scratch/x2093a05/.conda/envs/notebook/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/scratch/x2093a05/.conda/envs/notebook/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


187/187 - 7s - loss: 0.1123 - mse: 0.1110
Train on 75 samples


...

27/27 - 7s - loss: 0.0383 - mse: 0.0370
Train on 21 samples


/scratch/x2093a05/.conda/envs/notebook/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/scratch/x2093a05/.conda/envs/notebook/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


21/21 - 5s - loss: 0.0454 - mse: 0.0442


/scratch/x2093a05/.conda/envs/notebook/lib/python3.7/site-packages/ipykernel_launcher.py:43: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


current_round : 4
RMSE:	0.315000
MAE:	0.315000
MSE:	0.099000
AUC:	0.702000
